# Crop Yield Prediction - Data Analysis & Modeling

This notebook provides comprehensive analysis of crop yield data and demonstrates the ML modeling pipeline.

## Contents
1. Data Loading & Exploration
2. Weather Data Analysis
3. Satellite Data Analysis
4. Feature Engineering
5. Model Training
6. Model Evaluation
7. Predictions & Visualizations

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# Add src to path
sys.path.append('../src')

from models.ml_models import CropYieldModels
from train_pipeline import CropPredictionPipeline

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

%matplotlib inline
%load_ext autoreload
%autoreload 2

print("Libraries loaded successfully!")

## 1. Data Loading & Exploration

In [ ]:
# Initialize pipeline
pipeline = CropPredictionPipeline(config_path='../config/config_template.yaml')

# Create synthetic data for demonstration
# Replace with: df = pipeline.load_and_merge_data() when you have real data
df = pipeline.create_synthetic_data(n_samples=2000)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Dataset summary statistics
df.describe()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing) > 0:
    print("Missing values:")
    print(missing)
else:
    print("No missing values!")

## 2. Yield Distribution Analysis

In [ ]:
# Yield distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['yield'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Yield (bushels/acre)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Crop Yields')
axes[0].axvline(df['yield'].mean(), color='red', linestyle='--', label=f'Mean: {df["yield"].mean():.1f}')
axes[0].legend()

# Box plot by year
df.boxplot(column='yield', by='year', ax=axes[1])
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Yield (bushels/acre)')
axes[1].set_title('Yield Distribution by Year')

plt.tight_layout()
plt.show()

## 3. Weather Data Analysis

In [ ]:
# Weather features correlation with yield
weather_cols = ['temp_mean', 'temp_max', 'temp_min', 'precipitation', 'humidity', 'gdd_cumulative', 'drought_index']
weather_cols = [c for c in weather_cols if c in df.columns]

correlations = df[weather_cols + ['yield']].corr()['yield'].drop('yield').sort_values(ascending=False)

plt.figure(figsize=(10, 6))
correlations.plot(kind='barh', color='steelblue')
plt.xlabel('Correlation with Yield')
plt.title('Weather Features Correlation with Crop Yield')
plt.tight_layout()
plt.show()

print("\nTop correlated features:")
print(correlations)

In [ ]:
# Scatter plots: Key weather features vs yield
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

plot_features = ['precipitation', 'gdd_cumulative', 'drought_index', 'temp_mean']
plot_features = [f for f in plot_features if f in df.columns]

for idx, feature in enumerate(plot_features[:4]):
    axes[idx].scatter(df[feature], df['yield'], alpha=0.5)
    axes[idx].set_xlabel(feature.replace('_', ' ').title())
    axes[idx].set_ylabel('Yield (bushels/acre)')
    axes[idx].set_title(f'Yield vs {feature.replace("_", " ").title()}')
    
    # Add trend line
    z = np.polyfit(df[feature], df['yield'], 1)
    p = np.poly1d(z)
    axes[idx].plot(df[feature], p(df[feature]), "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

## 4. Satellite Data Analysis

In [ ]:
# Satellite features correlation
satellite_cols = ['ndvi_mean', 'ndvi_max', 'evi_mean', 'soil_moisture_mean']
satellite_cols = [c for c in satellite_cols if c in df.columns]

if satellite_cols:
    sat_correlations = df[satellite_cols + ['yield']].corr()['yield'].drop('yield').sort_values(ascending=False)
    
    plt.figure(figsize=(10, 6))
    sat_correlations.plot(kind='barh', color='forestgreen')
    plt.xlabel('Correlation with Yield')
    plt.title('Satellite Features Correlation with Crop Yield')
    plt.tight_layout()
    plt.show()
    
    print("\nSatellite feature correlations:")
    print(sat_correlations)

## 5. Feature Correlation Heatmap

In [ ]:
# Correlation heatmap of all numeric features
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != 'year']  # Exclude year

plt.figure(figsize=(14, 12))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Model Training

In [ ]:
# Prepare features and target
X, y = pipeline.prepare_features_and_target(df)

print(f"Features: {X.shape}")
print(f"Target: {y.shape}")
print(f"\nFeature names: {list(X.columns)}")

In [ ]:
# Train all models
results = pipeline.train_and_evaluate(X, y)

## 7. Model Comparison

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame(results['model_comparison'])

# Display comparison table
print("\nModel Performance Comparison:")
print("="*80)
display(comparison_df)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# R² comparison
axes[0, 0].barh(comparison_df['model'], comparison_df['r2'], color='steelblue')
axes[0, 0].set_xlabel('R² Score')
axes[0, 0].set_title('Model R² Comparison (Higher is Better)')
axes[0, 0].grid(axis='x', alpha=0.3)

# RMSE comparison
axes[0, 1].barh(comparison_df['model'], comparison_df['rmse'], color='coral')
axes[0, 1].set_xlabel('RMSE')
axes[0, 1].set_title('Model RMSE Comparison (Lower is Better)')
axes[0, 1].grid(axis='x', alpha=0.3)

# MAE comparison
axes[1, 0].barh(comparison_df['model'], comparison_df['mae'], color='lightgreen')
axes[1, 0].set_xlabel('MAE')
axes[1, 0].set_title('Model MAE Comparison (Lower is Better)')
axes[1, 0].grid(axis='x', alpha=0.3)

# MAPE comparison
axes[1, 1].barh(comparison_df['model'], comparison_df['mape'], color='gold')
axes[1, 1].set_xlabel('MAPE (%)')
axes[1, 1].set_title('Model MAPE Comparison (Lower is Better)')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Feature Importance Analysis

In [ ]:
# Get feature importance from best models
best_models = ['random_forest', 'xgboost', 'lightgbm']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, model_name in enumerate(best_models):
    if model_name in pipeline.models.feature_importance:
        importance = pipeline.models.feature_importance[model_name].head(10)
        
        axes[idx].barh(importance['feature'], importance['importance'])
        axes[idx].set_xlabel('Importance')
        axes[idx].set_title(f'{model_name.replace("_", " ").title()}\nTop 10 Features')
        axes[idx].invert_yaxis()

plt.tight_layout()
plt.show()

## 9. Prediction Visualization

In [ ]:
# Get predictions from best model
best_model_name = results['best_model']
best_model = pipeline.models.models[best_model_name]

# Prepare test data
data = pipeline.models.prepare_data(X, y, test_size=0.2)
X_test = data['X_test']
y_test = data['y_test']

# Make predictions
y_pred = best_model.predict(X_test)
if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
    y_pred = y_pred.flatten()

# Actual vs Predicted plot
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot
axes[0].scatter(y_test, y_pred, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Yield')
axes[0].set_ylabel('Predicted Yield')
axes[0].set_title(f'Actual vs Predicted - {best_model_name.replace("_", " ").title()}')
axes[0].grid(alpha=0.3)

# Residual plot
residuals = y_test - y_pred
axes[1].scatter(y_pred, residuals, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Yield')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest Model: {best_model_name}")
print(f"R²: {results['best_r2']:.4f}")
print(f"RMSE: {results['best_rmse']:.2f}")

## 10. Save Models and Results

In [ ]:
# Save all results
pipeline.save_results(output_dir='../outputs')

print("✅ All models and results saved successfully!")

## Summary

This notebook demonstrated:
1. Loading and exploring crop yield data
2. Analyzing weather and satellite features
3. Training multiple ML models (Linear Regression, Random Forest, XGBoost, LightGBM, SVR, MLP)
4. Comparing model performance
5. Analyzing feature importance
6. Visualizing predictions

### Next Steps:
- Collect real data using the data collection scripts
- Experiment with additional features
- Try ensemble methods
- Implement LSTM for time-series modeling
- Add SHAP values for model interpretation